# Module 7: PID Control for Line Tracking

Now we have reached the point where we will actually have the Jetbot travel around the lane based upon our previous module. This will be accomplished via a PD Controller.

This module should follow Module 6: Mid-Line Detection

Below is our normal starter code for initializing all variables and packages. An image should appear to ensure it is working properly.

In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
import ipywidgets as widgets
from scipy.signal import medfilt
from IPython.display import display, Image, clear_output
import time
import threading

robot = Robot()
# Initialize our camera
camera = Camera.instance()

# Save image data (modified to be numpy array)
image = np.array(camera.value)

jpeg_image = bgr8_to_jpeg(image)

# Display image within Jupyter Notebook
display(Image(data=jpeg_image))

## Method

A PID Controller stands for Proportional-Integral-Derivative, which is used to describe the various operations for a closed-feedback loop. It works by taking the error between the state of the system and the desired position. In this case, our state of the system is defined as the angle of the line created between the middle of the camera and the median of the points near the top. Then the desired state is a 90 degree line.

We will utilize our previous code from the previous Module 6, with added functions to calculate the slope and controller output.

## Implementation

Here are the steps we will follow to accomplish our task.

1. Midpoint Detection

    This is based on our previous module, where we detected the yellow lane markings. Make sure to copy the functions from that program, including with your tuned variables.

2. Slope Calculation

    Now that we have the midpoints of the lane markings, the slope and angle of the line is calculated using trigonometric principles. This also includes various filtering techniques to remove outliers and noisy data.

3. PID Tuning

    This is the main bulk of this module. With our error now calculated, we must find values for the PD controller that work best at the line tracking. While there are various techniques for calculating these tuning values, we will attempt to find them with trial and error.

The main activity will involve modifying the PD values to find the best values that allow the robot to follow the yellow lane markings. A good procedure for this would be to start by tuning the Proportional parameter with the Derivative Parameter set to 0, then once you find a good Proportional value, then tuning the Derivative parameter to improve the performance, i.e. making the motion smoother.

### Activity

Let's start by putting your HSV range values that you found in Module 6 below.

In [ ]:
# TODO: Put in your HSV values you found in Module 6
lower_yellow = (0, 20, 100)
upper_yellow = (75, 100, 255)

The next block is the main functions that we had in Module 6. If you made any changes to the functions themselves, here is where you would update them. There is also an added function called <i>compute_slope_error</i> that takes the midpoints we found and calculate the slope that the <b><span style="color:#154734">JetBot</span></b> should follow.

In [ ]:
# Brighten Gamma of Image
def adjust_gamma(image, gamma=1.2):
    invGamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** invGamma * 255 for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(image, table)

# Filter based on inter-quartile range (remove outliers)
def iqr_filter(x_list):
    x = np.array(x_list)
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return [val for val in x if lower <= val <= upper]

# Calculate midpoints based on masked image
def lane_midpoint(masked_image):
    midline_points = []
    left_points = []
    right_points = []
    h, w = masked_image.shape[:2]

    for y in range(0, h, 4):
        x_candidates = [x for x in range(w) if masked_image[y, x] > 0]
        if len(x_candidates) >= 3:
            x_candidates = iqr_filter(x_candidates)
            left_x = min(x_candidates)
            right_x = max(x_candidates)
            center_x = int(np.mean(x_candidates))

            midline_points.append((center_x, y))
            left_points.append((left_x, y))
            right_points.append((right_x, y))

    if midline_points:
        x_vals = [x for x, _ in midline_points]
        x_smooth = medfilt(x_vals, kernel_size=5)
        midline_points = list(zip(x_smooth, [y for _, y in midline_points]))
    
    return midline_points, left_points, right_points

# Isolate Yellow Lane Marking
def get_yellow_lane(image, apply_gamma=True):

    # Brighten Images
    if apply_gamma:
        image = adjust_gamma(image, gamma=1.2)

    # Blur and Convert to HSV Color Space
    blur = cv2.bilateralFilter(image, 9, 75, 75)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

    # HSV color masking
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Morphology to clean specks (Dilation and Erosion)
    kernel = np.ones((5, 5), np.uint8)
    opened = cv2.morphologyEx(mask_yellow, cv2.MORPH_OPEN, kernel, iterations=1)
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel, iterations=1)

    # Remove small blobs
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    clean_mask = np.zeros_like(closed)
    for cnt in contours:
        if cv2.contourArea(cnt) > 150:
            cv2.drawContours(clean_mask, [cnt], -1, 255, thickness=cv2.FILLED)
    contour_mask = clean_mask

    # Edge detection
    yellow_mask_blur = cv2.GaussianBlur(contour_mask, (5, 5), 0)
    sobelx = cv2.Sobel(yellow_mask_blur, cv2.CV_64F, 1, 0, ksize=3)
    sobelx = np.absolute(sobelx)
    if np.max(sobelx) > 0:
        sobelx = np.uint8(255 * sobelx / np.max(sobelx))
    else:
        sobelx = np.zeros_like(sobelx, dtype=np.uint8)
    _, edge_mask = cv2.threshold(sobelx, 50, 255, cv2.THRESH_BINARY)

    return mask_yellow, edge_mask

# Calculate error of slope from desired vertical line
def compute_slope_error(image, mid_pts, num_points=5):
    h, w = image.shape[:2]
    bottom_center = (w // 2, h - 1)
    if len(mid_pts) < num_points * 2:
        return None  # Not enough points

    # Takes sample of various points near top
    top_pts = mid_pts[:num_points]

    x_top_avg = np.median([pt[0] for pt in top_pts])
    y_top_avg = np.median([pt[1] for pt in top_pts])

    if bottom_center[1] == y_top_avg:
        return None  # Prevent division by zero

    # Calculate slope of line
    slope = (bottom_center[0] - x_top_avg) / (bottom_center[1] - y_top_avg)
    slope_error = 1/slope
    dx = bottom_center[0] - x_top_avg
    dy = bottom_center[1] - y_top_avg

    # Calculates angle of slope
    angle_rad = np.arctan2(dy, dx)
    angle_deg = np.degrees(angle_rad)

    # Calculating error from vertical
    slope_error_angle = angle_deg - 90

    out_img = image.copy()
    # Overlay line and text
    cv2.line(out_img, (int(x_top_avg), int(y_top_avg)), (int(bottom_center[0]), int(bottom_center[1])), (0, 255, 255), 2)
    cv2.putText(out_img, f"Angle: {slope_error_angle:.2f} deg", (30, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    return out_img, slope_error_angle

The next block are some custom classes that help to organize our data. They allow for us to store relevant information together, as well as functions that utilize that information in an easier way. This includes a SlopeFilter, which allows us to store previous slope results and filter outliers, and a PD Controller, which allows us to easily calculate the controller output and store previous errors.

In [ ]:
from collections import deque

# Filters out outlier slopes
class SlopeFilter:
    def __init__(self, max_len=3, max_angle=40):
        self.buffer = deque(maxlen=max_len)
        self.max_angle = max_angle

    def update(self, new_val):
        if new_val is None:
            return self.get_median()

        if abs(new_val) <= self.max_angle:
            self.buffer.append(new_val)
        
        return self.get_median()

    def get_median(self):
        if not self.buffer:
            return 0.0  # default if empty
        return float(np.median(self.buffer))

# PD Controller Class
class PIDController:
    def __init__(self, Kp=0.4, Kd=0.2):
        self.Kp = Kp
        self.Kd = Kd
        self.last_error = 0.0
        self.last_time = time.time()

    def compute(self, error):
        current_time = time.time()
        dt = current_time - self.last_time
        de = (error - self.last_error) / dt if dt > 0 else 0.0

        output = self.Kp * error + self.Kd * de

        self.last_error = error
        self.last_time = current_time
        return output

Here is the block where we interface and run the previous functions. After you run the next block, a few different widgets should appear that show the live feed, buttons to turn on and off, showing the slope and controller output, and widgets to update the P and D parameters for the controller in real time.

Run the following code, and try to tune the P and D parameters to effectively traverse the yellow lane markings.

In [ ]:
# Widgets (live feed, slope error, control output, P and D parameters)
image = widgets.Image(format='jpeg', width=300, height=300)
slope_label = widgets.Label(value="Slope Error: ---")
control_label = widgets.Label(value="Control Output: ---")
kp_input = widgets.FloatText(value=0.0035, description='Kp')
kd_input = widgets.FloatText(value=0.001, description='Kd')

# Start and stop buttons to control robot
go_button = widgets.Button(description="Go", button_style='success')
stop_button = widgets.Button(description="Stop", button_style='danger')
robot_enabled = True  # flag to control robot movement

# Button Callbacks
def on_go_clicked(b):
    global robot_enabled
    robot_enabled = True

def on_stop_clicked(b):
    global robot_enabled
    robot_enabled = False
    robot.stop()

go_button.on_click(on_go_clicked)
stop_button.on_click(on_stop_clicked)

# Displaying the widgets
display(widgets.VBox([
    image,
    widgets.HBox([go_button, stop_button]),
    slope_label,
    control_label,
    kp_input,
    kd_input
]))

# Class objects
filter = SlopeFilter(3, 40)
pid = PIDController(Kp=kp_input.value, Kd=kd_input.value)

# Driving Speeds for safe handling
base_speed = 0.1  # forward speed
max_turn = 0.08   # max allowed turn difference

stop_event = threading.Event()

# update loop
def update_image():
    while not stop_event.is_set():
        try:
            frame = camera.value
            yellow_mask, edge_mask = get_yellow_lane(frame)
            midline_points, left_points, right_points = lane_midpoint(edge_mask)
            overlay = frame.copy()
            for (x, y) in midline_points:
                cv2.circle(overlay, (int(x), int(y)), 2, (0, 255, 0), -1)
            for (x, y) in left_points:
                cv2.circle(overlay, (int(x), int(y)), 2, (255, 0, 0), -1)
            for (x, y) in right_points:
                cv2.circle(overlay, (int(x), int(y)), 2, (0, 0, 255), -1)
            result = compute_slope_error(overlay, midline_points, 1)

            if result is not None:
                out_img, slope_error_angle = result
                image.value = bgr8_to_jpeg(out_img)

                filtered_slope = filter.update(slope_error_angle)
                slope_label.value = f"Slope Error: {filtered_slope:.2f}°"

                # Update PID parameters from sliders
                pid.Kp = kp_input.value
                pid.Kd = kd_input.value

                control = pid.compute(filtered_slope)
                control = max(min(control, max_turn), -max_turn)
                control_label.value = f"Control Output: {control:.4f}"

                if robot_enabled:
                    # Turning Mechanism
                    left_speed = base_speed
                    right_speed = base_speed

                    right_turn = right_speed - control   # turning right
                    left_turn = left_speed + control     # turning left

                    left_speed = max(left_speed, left_turn)
                    right_speed = max(right_speed, right_turn)

                    robot.set_motors(left_speed, right_speed)
                else:
                    robot.stop()
            else:
                slope_label.value = "Slope Error: ---"
                control_label.value = "Control Output: ---"
                if robot_enabled:
                    robot.set_motors(base_speed, base_speed)

        except Exception as e:
            slope_label.value = f"Error: {str(e)}"
            control_label.value = "Control Output: ---"
            robot.stop()

        time.sleep(0.05)

# Starts the Feed
camera.start()
thread = threading.Thread(target=update_image, daemon=True)
thread.start()


Some general tips when tuning.

- Begin by setting the D parameter to 0, and only tuning the P parameter until the <b><span style="color:#154734">JetBot</span></b> follows the line with a little bit of oscillations going back and forth on either side.

- Once you find a good value, try tuning your D parameter to reduce these oscillations, while still maintaining tracking. Generally, you should keep you D value less than your P value.

- Check to make sure it works in various settings, including both straight and turning roads. Don't just tune for one setting.

In [ ]:
stop_event.set()
thread.join()
stop_event.clear()
camera.stop()